In [13]:
#numpy moved irr function to numpy_financial
!pip install numpy_financial

In [14]:
#Import libraries
import pandas as pd
import numpy as np
import numpy_financial as npf
from typing import List

#Set pandas display options
pd.options.display.float_format = '{:20,.2f}'.format

### Cashflow Waterfall Model

In [15]:
#Define Mortgage Class
class Mortgage:
    #This class is initialized with outstanding principal left on the mortgage,
    #the coupon rate of the mortgage, and the number of months left (term).
    def __init__(self, outstanding_principal, coupon_rate, term):
        self.outstanding_principal = outstanding_principal
        self.coupon_rate = coupon_rate        
        self.term = int(term)
        self.payment = self.amortization() #We use an amortization function to define the monthly payments
        self.cashflow_df = self.cashflows() #We convert the mortgages to a dataframe with cashflows
    
    
    #Amortization function uses the monthly coupon rate, term number, and outstanding
    #principal to calculate the monthly payments required
    def amortization(self):
        x = (1 + (self.coupon_rate/12)) ** self.term
        return self.outstanding_principal * (((self.coupon_rate/12) * x) / (x - 1))
    
    
    #This function takes all the information we have on the mortgage and builds
    #a dataframe that has the periods, starting_balance, interest, principal, cashflow,
    #ending_balance, and effective coupon
    def cashflows(self):
        cashflow = []
        principal = self.outstanding_principal
        for period in range(self.term):
            interest_payment = principal * (self.coupon_rate / 12) #Calculate current interest payment
            principal_paid = self.payment - interest_payment #principal payment is amortized payment - interest payment
            
            #Input values into a temporary dictionary to be added to the cashflow list
            temp_dict = {'period': period + 1, 
                         'starting_balance': principal, 
                         'interest': interest_payment, 
                         'principal': principal_paid,
                         'cashflow': self.payment,
                         'ending_balance': principal - principal_paid,
                         'effective_coupon': self.coupon_rate * 100}
            
            cashflow.append(temp_dict)
            principal -= principal_paid
        return pd.DataFrame(cashflow).set_index('period') #return cashflow list as dataframe

In [16]:
#Define Bond Class
class Bond:
    #This class is initialized with a list of classes, list of principal balances, and a list of Mortgage objects.
    #A Bond can be made up of any number of classes as long as their are the same number of principal_balances
    #It is also required that the classes and principal_balances are input in the same order for correct naming.
    def __init__(self, classes: List[str], principal_balances: List[int], mortgages: List[Mortgage]):
        if len(classes) != len(principal_balances):
            raise ValueError("The lengths of 'classes' and 'principal_balances' must be the same.")
        
        self.classes = classes
        self.principal_balances = principal_balances
        self.mortgages = mortgages
        self.aggregated_cashflow = self.aggregate_cashflows() #A dataframe with all mortgage cashflows aggregated
        self.bond_df = self.waterfall() #A dataframe with the cashflow waterfall model implemented
        self.bond_WALs = self.weighted_average_life() #A dictionary containing the WAL for each bond class
        self.bond_total_cashflows = self.total_cashflows() #A dictionary containing the total cashflow for each bond class
        self.bond_IRRs = self.internal_rate_of_return() #A dictionary containing the IRR for each bond class
    
    
    #This function goes through each mortgage and adds all values by index which is period and by column
    def aggregate_cashflows(self):
        mortgage_sum = self.mortgages[0].cashflow_df

        for i in range(1, len(self.mortgages)):
            mortgage_sum = mortgage_sum.add(self.mortgages[i].cashflow_df, fill_value=0)
        
        #The effect coupon rate can then be calculated using the sum of all interest divided by the starting balance
        #This is then annualized and made to be represented as a percentage
        mortgage_sum['effective_coupon'] = mortgage_sum['interest']/mortgage_sum['starting_balance'] * 12 * 100
        
        return mortgage_sum
    
    
    #This function creates the waterfall model using the aggregated cashflow, classes, and principal_balances
    def waterfall(self):
        #start by creating a list of column names for each class in classes
        columns = ['period']
        for class_name in self.classes:
                columns.append(f'{class_name}_balance')
                columns.append( f'{class_name}_interest')
                columns.append(f'{class_name}_principal')
                columns.append(f'{class_name}_cashflow')
        cashflow_df = pd.DataFrame(columns=columns) #initialize empty dataframe with the column list
        
        #Iterate through the aggregated_cashflow
        principal_balances = self.principal_balances[:]
        for period, row in self.aggregated_cashflow.iterrows():
            interest_payment = row['interest'] #Current period total interest payment
            principal_payment = row['principal'] #Current period total principal payment
            starting_balance = row['starting_balance'] #Current period starting balance
            
            temp_dict = {'period': period}  #Iterate through each bond class
            for i in range(len(principal_balances)): 
                #If the current principal of the bond is greater than 0 we pay interest and principal
                if principal_balances[i] > 0:
                    #total interest paid for the current bond class
                    interest_paid = (principal_balances[i] / starting_balance) * interest_payment
                    #principal paid is the minimum of the current total principal payment and the principal balance
                    principal_paid = min(principal_payment, principal_balances[i]) 
                    principal_balances[i] -= principal_paid #pay principal to principal balance
                    #reduce principal payment by principal paid, if left over will be passed to next bond class
                    principal_payment -= principal_paid
                    #Add values to temporary dictionary until each class represented
                    temp_dict.update({
                        f'{self.classes[i]}_balance': principal_balances[i],
                        f'{self.classes[i]}_interest': interest_paid,
                        f'{self.classes[i]}_principal': principal_paid,
                        f'{self.classes[i]}_cashflow': principal_paid + interest_paid
                    })
                #Otherwise the bond class is fully paid off and nothing is paid
                else:
                    temp_dict.update({
                        'period': period,
                        f'{self.classes[i]}_balance': 0.0,
                        f'{self.classes[i]}_interest': 0.0,
                        f'{self.classes[i]}_principal': 0.0,
                        f'{self.classes[i]}_cashflow': 0.0
                    })
            #Add temporary dictionary from current period to cashflow_df
            cashflow_df =  pd.concat([cashflow_df, pd.DataFrame([temp_dict])], ignore_index=True)
        return cashflow_df
    
    
    #This function calculates the weighted average life for each bond using the classes and bond_df fields
    def weighted_average_life(self):
        wal = {}
        #Iterate through each class
        for i in range(len(self.classes)):
            #Get parts of bond_df where the current class's principal is greater than 0
            paid_period = self.bond_df.loc[self.bond_df[f'{self.classes[i]}_principal'] >= 0]
            wal.update({
                #We calculate the WAL using the sum of principal * (period/12) which is then divided by the starting balance
                f'Bond Class {self.classes[i]} WAL': round(sum(paid_period[f'{self.classes[i]}_principal'] * (paid_period['period']/12)) / self.principal_balances[i] , 2)
            })
        return wal
    
    
    #This function calculates the total cashflow for each bond using the classes and bond_df fields
    def total_cashflows(self):
        cashflows = {}
        #Iterate through each class
        for i in range(len(self.classes)):
            cashflows.update({
                #Sum all cashflow for the given bond class
                f'Bond Class {self.classes[i]} Total Cashflow': round(sum(self.bond_df[f'{self.classes[i]}_cashflow']), 2)
            })
        return cashflows
    
    
    #This function calculates the IRR for each bond using the classes and bond_df fields
    #This function also utilizes the numpy_finance library and the irr function within this library
    def internal_rate_of_return(self):
        irrs = {}
        #Iterate through each class
        for i in range(len(self.classes)):
            #Get list of cashflows with the first value being the negative starting principal of the bond class
            cashflow_list = np.insert(self.bond_df.loc[self.bond_df[f'{self.classes[i]}_cashflow'] != 0, f'{self.classes[i]}_cashflow'].values, 0, -(self.principal_balances[i]))
            irrs.update({
                #Pass irr function the cashflow_list
                #Annualize IRR by multiplying by 12 and multiply by 100 to get as a percent
                f'Bond Class {self.classes[i]} IRR': npf.irr(cashflow_list) * 12 * 100
            })
        return irrs

In [17]:
#Get Loan data from csv into a pandas dataframe
loan_data = pd.read_excel("")#Change to your file of loans
loan_data #Load loan data to make sure everything was input and loaded correctly

,ID,Cut Off Date Balance,Gross Coupon,Remaining Amortization
0,1001,5000000,5.50,180
1,1002,4000000,6.88,360
2,1003,6500000,6.88,360
3,1004,7750000,5.12,180
4,1005,10000000,6.88,360
5,1006,2250000,6.25,180
6,1007,7000000,6.62,180
7,1008,8000000,5.38,360
8,1009,12500000,7.00,360
9,1010,6750000,5.88,360


In [18]:
mortgages = []

#Create list of mortgage objects using the loan data
for _, row in loan_data.iterrows():
    mortgage_instance = Mortgage(
        outstanding_principal=row['Cut Off Date Balance'],
        coupon_rate=row['Gross Coupon']/100,
        term=row['Remaining Amortization']
    )
    mortgages.append(mortgage_instance)

In [19]:
#In this example wewill,
#Create bonds with classes A, B, C which correspond to 70000000, 20000000, and 10000000 and the list of mortgages
#These can be whatever class names, however many classes, and whatever principal balances, just ensure that the number
#of classes is the same as the number of principal
bonds = Bond(classes=['A', 'B', 'C'] , principal_balances=[70000000, 20000000, 10000000], mortgages=mortgages)

C:\Users\Maxim\AppData\Local\Temp\ipykernel_30044\3468015978.py:80: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cashflow_df =  pd.concat([cashflow_df, pd.DataFrame([temp_dict])], ignore_index=True)


In [20]:
#View aggregated cashflow from all mortgages 
bonds.aggregated_cashflow

,starting_balance,interest,principal,cashflow,ending_balance,effective_coupon
period,,,,,,
1,"100,000,000.00","506,458.33","199,790.35","706,248.68","99,800,209.65",6.08
2,"99,800,209.65","505,479.09","200,769.60","706,248.68","99,599,440.06",6.08
3,"99,599,440.06","504,494.98","201,753.70","706,248.68","99,397,686.36",6.08
4,"99,397,686.36","503,506.00","202,742.68","706,248.68","99,194,943.68",6.08
5,"99,194,943.68","502,512.12","203,736.56","706,248.68","98,991,207.12",6.08
...,...,...,...,...,...,...
356,"1,794,900.71","9,470.48","355,212.16","364,682.63","1,439,688.56",6.33
357,"1,439,688.56","7,596.53","357,086.10","364,682.63","1,082,602.45",6.33
358,"1,082,602.45","5,712.57","358,970.07","364,682.63","723,632.39",6.33


In [21]:
#View cashflow waterfall model
bonds.bond_df.to_excel('Waterfall Model Output.xlsx')
bonds.bond_df

,period,A_balance,A_interest,A_principal,A_cashflow,B_balance,B_interest,B_principal,B_cashflow,C_balance,C_interest,C_principal,C_cashflow
0,1,"69,800,209.65","354,520.83","199,790.35","554,311.18","20,000,000.00","101,291.67",0.00,"101,291.67","10,000,000.00","50,645.83",0.00,"50,645.83"
1,2,"69,599,440.06","353,531.78","200,769.60","554,301.38","20,000,000.00","101,298.20",0.00,"101,298.20","10,000,000.00","50,649.10",0.00,"50,649.10"
2,3,"69,397,686.36","352,537.81","201,753.70","554,291.51","20,000,000.00","101,304.78",0.00,"101,304.78","10,000,000.00","50,652.39",0.00,"50,652.39"
3,4,"69,194,943.68","351,538.88","202,742.68","554,281.56","20,000,000.00","101,311.41",0.00,"101,311.41","10,000,000.00","50,655.71",0.00,"50,655.71"
4,5,"68,991,207.12","350,534.98","203,736.56","554,271.54","20,000,000.00","101,318.09",0.00,"101,318.09","10,000,000.00","50,659.05",0.00,"50,659.05"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,356,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,439,688.56","9,470.48","355,212.16","364,682.63"
356,357,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,082,602.45","7,596.53","357,086.10","364,682.63"
357,358,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"723,632.39","5,712.57","358,970.07","364,682.63"
358,359,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"362,768.28","3,818.53","360,864.11","364,682.63"


In [22]:
#View Weighted Average Life for each bond class
bonds.bond_WALs

{'Bond Class A WAL': 10.33,
 'Bond Class B WAL': 24.52,
 'Bond Class C WAL': 28.84}

In [23]:
#View Total Cashflows for each bond class
bonds.bond_total_cashflows

{'Bond Class A Total Cashflow': 114361173.93,
 'Bond Class B Total Cashflow': 50453109.97,
 'Bond Class C Total Cashflow': 27953352.84}

In [24]:
#View Internal Rate of Return for each bond class
bonds.bond_IRRs

{'Bond Class A IRR': 6.121032495558776,
 'Bond Class B IRR': 6.171282866905958,
 'Bond Class C IRR': 6.180844317905976}